In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
crypto = pd.read_csv('../input/CryptocoinsHistoricalPrices.csv')
crypto.head()

FileNotFoundError: File b'Kaggle/input/CryptocoinsHistoricalPrices.csv' does not exist

In [ ]:
len(crypto)

In [ ]:
crypto.head()

In [ ]:
type(crypto['Date'][0])
#its a stringcrypt

In [ ]:
#Drop mising values
crypto.dropna(axis=0, inplace=True)
crypto.head()

In [ ]:
#Extracting year from variable date
crypto['Year']=crypto['Date'].apply(lambda x: x.split('-')[0])

In [ ]:
crypto.tail()

In [ ]:
#few rows have empty market cap values, we will remove it
df= crypto[crypto['Market.Cap']!='-']

In [ ]:
len(df)

In [ ]:
#Checking null values in columns
#sns.heatmap(crypto.isnull(), yticklabels=False, cbar=False, cmap='viridis')
df.isnull().sum()

In [ ]:
#columns glossary
#imp columns -> delta - change in currency opening and closing price, we want more positive values
#market cap -> transactions happened over a day -> price * no of transactions
#Volume -> amount of traffic

In [ ]:
df['Market.Cap'] = df['Market.Cap'].str.replace(',', '')
df['Market.Cap']=df['Market.Cap'].astype(float)

In [ ]:
#Sorting data on Date column
df.Date = pd.to_datetime(df.Date)
df.sort_values('Date')

In [ ]:
#taking Jul 2015 - Jul 2018 data to check recent cryptocurrencies in market
mask = (df['Date'] >= '2015-07-01') & (df['Date'] <= '2018-07-31')
df = df.loc[mask]
df.head()
len(df)

In [ ]:
df.head()

In [ ]:
#checking widely present currencies by checking market cap
temp_df= df.sort_values('Market.Cap', ascending=False)

In [ ]:
#By Market Cap, top 10 coins
coins_type2=temp_df.groupby('coin').sum().sort_values(by='Market.Cap',ascending=False).head(10)
coins_type2

In [ ]:
#we see that the currencies here are the topmost currencies in terms of market cap and we will be just focusing on them as of now

In [ ]:
#We know BTC has the highest share of market, but lets look at the promising cryptos now

In [ ]:
top_coins_name=list(coins_type2.index)

In [ ]:
df_top = df[df['coin'].isin(top_coins_name)]

In [ ]:
len(df_top)

In [ ]:
#Viz for exploratory data analysis

In [ ]:
df_top.head()

In [ ]:
import datetime
#df_top['Month']=df_top.Date.month()
df_top['Month']=pd.DatetimeIndex(df_top['Date']).month

In [ ]:
df_top.head()
#df_top.info()

In [ ]:
#Volume comparison
df_top.info()

**Top 10 Cryptocurrencies - Exploration**

In [ ]:
#barplot
plt.figure(figsize=(12,6))
sns.set(style="whitegrid")
sns.barplot(x='coin', y='Market.Cap', data=df_top, estimator=np.mean)

In [ ]:
#Summary of data
df_top['Volume'] = df_top['Volume'].str.replace(',', '')
df_top['Volume']=df_top['Volume'].astype(float)
df_top.describe()

In [ ]:
#Changing data types of Open and Close
df_top['Open']=pd.to_numeric(df_top['Open'])
df_top['Close']=df_top['Close'].astype(float)
df_top.info()

In [ ]:
#Histogram of variables
df_top.hist(figsize = (16,8),color='c',linestyle='solid', linewidth=1)

In [ ]:
#Boxplot for outliers
plt.figure(figsize=(12,6))
ax = sns.boxplot(data=df_top[['Open','Close','High','Low','Volume','Market.Cap','Delta']],orient="h")

In [ ]:
df_top.describe()

In [ ]:
len(df_top)

In [ ]:
df_top.info()

In [ ]:
#Transforming variables
df_top['Open_log'] = np.log(df_top['Open'])
df_top['Close_log'] = np.log(df_top['Close'])
df_top['High_log'] = np.log(df_top['High'])
df_top['Low_log'] = np.log(df_top['Low'])
df_top['Market.Cap_log'] = np.log(df_top['Market.Cap'])
df_top['Volume_log'] = np.log(df_top['Volume'])

df_top.hist(figsize = (24,12),color='c',linestyle='solid', linewidth=1)

In [ ]:
plt.figure(figsize=(12,6))
axx = sns.boxplot(data=df_top[['Open_log','Close_log','High_log','Low_log','Volume_log','Market.Cap_log']],orient="h")

In [ ]:
#Correlation plot
plt.figure(figsize=(12,6))
sns.heatmap(df_top.corr(), annot = True)

#Since, all the variables except Delta has very high correlation with close price, hence, we can choose one of these variables
#to build model for predicting close price of the coin

In [ ]:
#Scatter plot of variables with Close price
#sns.regplot(x=df_top[("High","Low","Volume","Market.Cap","Open","Delta")], y=df_top["Close"], fit_reg=False)
#sns.plt.show()
plt.figure(figsize=(12,6))
pp = sns.pairplot(data=df_top,
                  y_vars=['Close'],
                  x_vars=['High', 'Low', 'Volume','Market.Cap','Open','Delta'])
#As shown in the plots, Delta doesn't have any significant linear correlation with Closing price. 
#Rest all the variables (except Volume) have very high correlation (~1, linear relationship) with closing price. 
#Hence, we can take any of these highly correlated variables to build our model

In [ ]:
#Scatter plot of variables with Close price
#sns.regplot(x=df_top[("High","Low","Volume","Market.Cap","Open","Delta")], y=df_top["Close"], fit_reg=False)
#sns.plt.show()
plt.figure(figsize=(12,6))
pp = sns.pairplot(data=df_top,
                  y_vars=['Close_log'],
                  x_vars=['Open_log','High_log','Low_log','Volume_log','Market.Cap_log'])
#As shown in the plots, Delta doesn't have any significant linear correlation with Closing price. 
#Rest all the variables (except Volume) have very high correlation (~1, linear relationship) with closing price. 
#Hence, we can take any of these highly correlated variables to build our model

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import zscore
#df_num =df_top[numeric_cols].apply(zscore)
df_top['High_std'] =df_top[['High']].apply(zscore)
df_top['Low_std'] =df_top[['Low']].apply(zscore)
df_top['Volume_std'] =df_top[['Volume']].apply(zscore)
df_top['Market.Cap_std'] =df_top[['Market.Cap']].apply(zscore)
df_top['Open_std'] =df_top[['Open']].apply(zscore)
df_top['Close_std'] =df_top[['Close']].apply(zscore)


In [ ]:
df_top.head()

In [ ]:
#Scatter plot
plt.figure(figsize=(18,10))
pp = sns.pairplot(data=df_top,
                  y_vars=['Close_std'],
                  x_vars=['Open_std','High_std','Low_std','Volume_std','Market.Cap_std'])

**Buy and Hold Strategy for top 4 coins**

In [ ]:
# Transforming top 4 coins data for Closing price and sorting by date
coins = ['BTC','ETH','BCH','XRP']
df_top4 = df_top[df_top.coin.isin(coins)]

#Filtering data for latest 1 year as BCH and ETH are new coins
mask = (df_top4['Date'] >= '2017-08-02') & (df_top4['Date'] <= '2018-07-31')
df_top4 = df_top4.loc[mask]

df_top4_close = df_top4[['Date','Close','coin']].sort_values(by = ['Date'])
df_top4_close = df_top4_close.pivot(index='Date', columns='coin', values='Close')
df_top4_close.head()

In [ ]:
#Closing price plot for top 4 coins
df_top4_close.plot(grid=True, figsize=(15, 10)).axhline(y = 1, color = "black", lw = 2)

In [ ]:
df_return = df_top4_close.apply(lambda x: x / x[0])
df_return.head()

In [ ]:
df_return.plot(grid=True, figsize=(15, 10)).axhline(y = 1, color = "black", lw = 2)

**What is the percentage increase?**

In [ ]:
df_perc = df_return.tail(1) * 100
ax = sns.barplot(data=df_perc)
df_perc

**How many coins could we bought for $1000?**

In [ ]:
budget = 1000 # USD
df_coins = budget/df_top4_close.head(1)

ax = sns.barplot(data=df_coins)
df_coins

**How much money would we make?**

In [ ]:
df_profit = df_return.tail(1) * budget

ax = sns.barplot(data=df_profit)
df_profit

**Model Building**

> **For BTC**

In [ ]:
#For BTC
#linear model for BTC using log trasnformed variables
btc_table= df_top[df['coin']=='BTC']
btc_table
X=btc_table[['Open_log', 'High_log', 'Low_log', 'Volume_log', 'Market.Cap_log']]
y=btc_table['Close_log']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_test.head()


In [ ]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
lm.fit(X_train, y_train)
print(lm.intercept_) #coefficients


In [ ]:
cdf=pd.DataFrame(lm.coef_,X.columns, columns=['Coeff'])
np.exp(cdf)

#market cap and volume is not playing a deciding factor here

In [ ]:
predictions=lm.predict(X_test)
y_test = np.exp(y_test)
predictions = np.exp(predictions)
plt.scatter(y_test, predictions)


In [ ]:
sns.distplot((y_test- predictions))
#we see it shows a linear trend and distribution is uniform with few outliers and since price is low , we cant be very sure whether it can be used as a good predictive model

In [ ]:
#Summary statistics for linear regression model
import statsmodels.api as sm
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

*Linear Model with multiple variables for BTC*

In [ ]:
#For BTC
#linear model for BTC with standardized variables
btc_table= df_top[df['coin']=='BTC']
btc_table
X=btc_table[['Open_std', 'High_std', 'Low_std', 'Volume_std', 'Market.Cap_std']]
y=btc_table['Close_std']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_test.head()

In [ ]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
lm.fit(X_train, y_train)
print(lm.intercept_) #coefficients


In [ ]:
lm.coef_


In [ ]:
cdf=pd.DataFrame(lm.coef_,X.columns, columns=['Coeff'])
cdf
#market cap is not playing a deciding factor here

In [ ]:
predictions=lm.predict(X_test)
plt.scatter(y_test, predictions)


In [ ]:
sns.distplot((y_test- predictions))
#we see it shows a linear trend and distribution is uniform with few outliers and since price is low , 
#we cant be very sure whether it can be used as a good predictive model

In [ ]:
#Summary statistics for linear regression model
import statsmodels.api as sm
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
#Evaluating model
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

*Linear Model with 'Open' variable only for BTC*

In [ ]:
#for bitcoin with only Open price as predictor variable
df_btc_open= pd.DataFrame({'Date':btc_table['Date'] , 'Values':btc_table['Open']})
df_btc_open.head()

In [ ]:
df_btc_open.sort_values(by='Date', inplace=True, ascending=True)
df_btc_open.tail()

In [ ]:
#df_btc_open.sort_values(by='Date', ascending=False, inplace=True)
plt.figure(figsize=(20,10))
df_btc_open.plot(x='Date', y='Values', kind='area')

In [ ]:
#the above analysis shows steep rise in BTC and then sudden decrease

In [ ]:
#Regression model for this case
X=pd.DataFrame(np.log(btc_table['Open']))
y=pd.DataFrame(np.log(btc_table['Close']))


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_test.head()

In [ ]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
lm.fit(X_train, y_train)
print(lm.intercept_) #coefficients


In [ ]:
#Predicting on test data
predictions=lm.predict(X_test)
plt.scatter(y_test, predictions)

In [ ]:
#Summary statistics for linear regression model
import statsmodels.api as sm
X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

*Random Forest Regressor model with multiple variables for BTC*

In [ ]:
#RANDOM FOREST model for prediction

X=btc_table[['Open', 'High', 'Low', 'Volume', 'Market.Cap']]
y=btc_table['Close']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


In [ ]:
#Random Forest Model for BTC
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=100,
                                 max_features=5, oob_score=True)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]


In [ ]:
#Feature importance plot
feat_importances = pd.Series(forest.feature_importances_, index=X_train.columns)
feat_importances.nlargest(5).plot(kind='barh')
#As observed, the closing price for BTC coin depends mainly on High, Low, Market Cap and Open values of the coin.

In [ ]:
#Predicting Closing price on testing data and calculating accuracy
predictions=forest.predict(X_test)
plt.scatter(y_test, predictions)

In [ ]:
#Calculating Rsq value for the model
from sklearn.metrics import r2_score
r2_score(y_test, predictions)

**Random Forest model for ETH****

In [ ]:
#for ethereum
eth_table= df_top[df['coin']=='ETH']
eth_table
df_eth_open= pd.DataFrame({'Date':eth_table['Date'] , 'Values':eth_table['Open']})
df_eth_open.head()
df_eth_open.sort_values(by='Date', inplace=True, ascending=True)
df_eth_open.tail()

In [ ]:
plt.figure(figsize=(20,10))
df_eth_open.plot(x='Date', y='Values', kind='area')
#Similar observation can be made for Etherium coin as BTC, steep rise and decline in the Open price during the 1st half of 2018.

In [ ]:
#RANDOM FOREST model for prediction

X=eth_table[['Open', 'High', 'Low', 'Volume', 'Market.Cap']]
y=eth_table['Close']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


In [ ]:
#Random Forest Model for ETH
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=100,
                                 max_features=5, oob_score=True)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]


In [ ]:
#Feature importance plot
feat_importances = pd.Series(forest.feature_importances_, index=X_train.columns)
feat_importances.nlargest(5).plot(kind='barh')
#As observed, the closing price for ETH coin depends mainly on High and Low values of the coin.

In [ ]:
#Predicting Closing price on testing data and calculating accuracy
predictions=forest.predict(X_test)
plt.scatter(y_test, predictions)

In [ ]:
#Calculating Rsq value for the model
from sklearn.metrics import r2_score
r2_score(y_test, predictions)

**Random Forest Regressor Model for BCH**

In [ ]:
#for Bitcoin Cash (BCH)
bch_table= df_top[df['coin']=='BCH']
bch_table
df_bch_open= pd.DataFrame({'Date':bch_table['Date'] , 'Values':bch_table['Open']})
df_bch_open.head()
df_bch_open.sort_values(by='Date', inplace=True, ascending=True)
df_bch_open.tail()

In [ ]:
plt.figure(figsize=(20,10))
df_bch_open.plot(x='Date', y='Values', kind='area')
#Distribution of Open price over time for BCH coin looks promising, however, there is steep rise and decline during the 
#2017 end and start of the year 2018 but the spread is still well distributed and average is close to $1000.

In [ ]:
#RANDOM FOREST model for prediction

X=bch_table[['Open', 'High', 'Low', 'Volume', 'Market.Cap']]
y=bch_table['Close']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


In [ ]:
#Random Forest Model for BCH
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=100,
                                 max_features=5, oob_score=True)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]


In [ ]:
#Feature importance plot
feat_importances = pd.Series(forest.feature_importances_, index=X_train.columns)
feat_importances.nlargest(5).plot(kind='barh')
#As observed, the closing price for BCH coin depends mainly on High and Low values of the coin.

In [ ]:
#Predicting Closing price on testing data and calculating accuracy
predictions=forest.predict(X_test)
plt.scatter(y_test, predictions)

In [ ]:
#Calculating Rsq value for the model
from sklearn.metrics import r2_score
r2_score(y_test, predictions)

**Random Forest Regressor Model for XRP**

In [ ]:
#for ripple
rip_table= df_top[df['coin']=='XRP']
rip_table.head()

In [ ]:
#for ripple
df_rip_open= pd.DataFrame({'Date':rip_table['Date'] , 'Values':rip_table['Open']})
df_rip_open.head()
df_rip_open.sort_values(by='Date', inplace=True, ascending=True)
df_rip_open.tail()

In [ ]:
plt.figure(figsize=(20,10))
df_rip_open.plot(x='Date', y='Values', kind='area')
#As shown in the plot, average value is lower as compared to the previous top 3 coins and there is a huge spike in the same 
#timeframe when the Open prices spiked for the top 3 coins. This coud be an effect of the market in general for all the coins 
#or, it could be an effect of the rise in BTC open prices as it is the 1st and most popular cryptocurrency. 

In [ ]:
#RANDOM FOREST model for prediction

X=rip_table[['Open', 'High', 'Low', 'Volume', 'Market.Cap']]
y=rip_table['Close']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


In [ ]:
#Random Forest Model for XRP
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=100,
                                 max_features=5, oob_score=True)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]


In [ ]:
#Feature importance plot
feat_importances = pd.Series(forest.feature_importances_, index=X_train.columns)
feat_importances.nlargest(5).plot(kind='barh')
#As seen from the plot, the Closing price depends on High and Low values rather than the Open and Market Cap of the coin.

In [ ]:
#Predicting Closing price on testing data and calculating accuracy
predictions=forest.predict(X_test)
plt.scatter(y_test, predictions)

In [ ]:
#Calculating Rsq value for the model
from sklearn.metrics import r2_score
r2_score(y_test, predictions)

In [ ]:
# from our top 4 cryptos viz here against their opening values
# Bitcoin, Ethereum and even Ripple have many updowns throughout the year and they started strong but then struggled
#whereas on the other hand BCH showed promising growth